In [1]:
#%%timeit
#Student name: Sally Moywaywa
#Cohort: 3
#loading required libraries
import pandas                    as pd 
import matplotlib.pyplot         as plt 
import seaborn                   as sns 
import random                    as rand
import statsmodels.formula.api   as smf 
from   sklearn.model_selection   import train_test_split 
from   sklearn.ensemble          import GradientBoostingRegressor
import sklearn.linear_model
from   sklearn.neighbors         import KNeighborsRegressor 
from   sklearn.preprocessing     import StandardScaler 
from   sklearn.linear_model      import LogisticRegression  
from   sklearn.metrics           import confusion_matrix         
from   sklearn.metrics           import roc_auc_score            
from   sklearn.neighbors         import KNeighborsClassifier 
from   sklearn.tree              import DecisionTreeClassifier      
from   sklearn.tree              import export_graphviz             
from   sklearn.externals.six     import StringIO           
from   IPython.display           import Image                    
import pydotplus                                     



/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
#loading the dataset
original_df = pd.read_excel('Apprentice_Chef_Dataset.xlsx')

In [3]:
#creating an empty list where the split email domains would be appended
list1 = []

# creating a loop that will loop over the email column, split the emails and append them to the empty list
for index, col in original_df.iterrows():
    
    email_split = original_df.loc[index, 'EMAIL'].split(sep = '@')
    
    list1.append(email_split)
    

#making the split email domain list into a dataframe
emails_df = pd.DataFrame(list1)

# linking the email_dataframe with apprentice chef DataFrame
# we re-read the oringinal dataset to prevent chances of linking the two dataframes multiple times
original_df = pd.read_excel("Apprentice_Chef_Dataset.xlsx")


# renaming the columns of the split email domain dataframe
emails_df.columns = ['NAME' ,'EMAIL_DOMAINS']

# linking the emails dataframe to the original dataframe
original_df = pd.concat([original_df, emails_df['EMAIL_DOMAINS']],
               axis = 1)

#creating the different email groups i.e professional, personal or junk as stated by the marketing team 

professional_emails = ['@mmm.com', '@amex.com','@apple.com', '@boeing.com',
                       '@caterpillar.com', '@chevron.com', '@cisco.com',
                       '@cocacola.com', '@disney.com', '@dupont.com', 
                       '@exxon.com', '@ge.org', '@goldmansacs.com', 
                       '@homedepot.com', '@ibm.com', '@intel.com', 
                       '@jnj.com', '@jpmorgan.com', '@mcdonalds.com',
                       '@merck.com', '@microsoft.com', '@nike.com', 
                       '@pfizer.com','@pg.com', '@travelers.com', 
                       '@unitedtech.com', '@unitedhealth.com',
                       '@verizon.com', '@visa.com', '@walmart.com']

personal_emails    = ['@gmail.com', '@yahoo.com', '@protonmail.com']

junk_emails        = ['@me.com', '@aol.com', '@hotmail.com',
                      '@live.com', '@msn.com', '@passport.com']

#creating an empty list where the email domains groups would be appended
list2 = []

# creating a loop that will loop over the email domains column, group the email domains and append them to the empty list

for domain in original_df['EMAIL_DOMAINS']:
        if '@' + domain in professional_emails: 
            list2.append('professional')
            
        elif '@' + domain in personal_emails:
            list2.append('personal')
            
        elif '@' + domain in junk_emails:
            list2.append('junk')
            

# adding the list to the original dataframe
original_df['EMAIL_DOMAINS'] = pd.Series(list2)



In [4]:
#Filling in missing values
# creating an imputation value that will be filled in place of missing data
fill = "N/A"


# imputing 'FAMILY_NAME'
original_df['FAMILY_NAME'] = original_df['FAMILY_NAME'].fillna(fill)



In [5]:
# setting outlier thresholds
TOTAL_MEALS_ORDERED_hi          = 300
UNIQUE_MEALS_PURCH_hi           = 8
CONTACTS_W_CUSTOMER_SERVICE_hi  = 10
AVG_TIME_PER_SITE_VISIT_hi      = 250
CANCELLATIONS_BEFORE_NOON_hi    = 6
EARLY_DELIVERIES_hi             = 5
EARLY_DELIVERIES_lo             = 1
LATE_DELIVERIES_hi              = 12.5
AVG_PREP_VID_TIME_hi            = 300
AVG_CLICKS_PER_VISIT_lo         = 7.5
TOTAL_PHOTOS_VIEWED_hi          = 750

# developing new columns for outlier values in the dataset



# TOTAL_MEALS_ORDERED
original_df['out_TOTAL_MEALS_ORDERED'] = 0
condition_hi = original_df.loc[0:,'out_TOTAL_MEALS_ORDERED'][original_df['TOTAL_MEALS_ORDERED'] > TOTAL_MEALS_ORDERED_hi]

original_df['out_TOTAL_MEALS_ORDERED'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


# UNIQUE_MEALS_PURCH
original_df['out_UNIQUE_MEALS_PURCH'] = 0
condition_hi = original_df.loc[0:,'out_UNIQUE_MEALS_PURCH'][original_df['UNIQUE_MEALS_PURCH'] < UNIQUE_MEALS_PURCH_hi]

original_df['out_UNIQUE_MEALS_PURCH'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


# CONTACTS_W_CUSTOMER_SERVICE
original_df['out_CONTACTS_W_CUSTOMER_SERVICE'] = 0
condition_hi = original_df.loc[0:,'out_CONTACTS_W_CUSTOMER_SERVICE'][original_df['CONTACTS_W_CUSTOMER_SERVICE'] > CONTACTS_W_CUSTOMER_SERVICE_hi]

original_df['out_CONTACTS_W_CUSTOMER_SERVICE'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


# AVG_TIME_PER_SITE_VISIT
original_df['out_AVG_TIME_PER_SITE_VISIT'] = 0
condition_hi = original_df.loc[0:,'out_AVG_TIME_PER_SITE_VISIT'][original_df['AVG_TIME_PER_SITE_VISIT'] > AVG_TIME_PER_SITE_VISIT_hi]

original_df['out_AVG_TIME_PER_SITE_VISIT'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


# CANCELLATIONS_BEFORE_NOON
original_df['out_CANCELLATIONS_BEFORE_NOON'] = 0
condition_hi = original_df.loc[0:,'out_CANCELLATIONS_BEFORE_NOON'][original_df['CANCELLATIONS_BEFORE_NOON'] > CANCELLATIONS_BEFORE_NOON_hi]

original_df['out_CANCELLATIONS_BEFORE_NOON'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


# EARLY_DELIVERIES
original_df['out_EARLY_DELIVERIES'] = 0
condition_hi = original_df.loc[0:,'out_EARLY_DELIVERIES'][original_df['EARLY_DELIVERIES'] > EARLY_DELIVERIES_hi]
condition_lo = original_df.loc[0:,'out_EARLY_DELIVERIES'][original_df['EARLY_DELIVERIES'] < EARLY_DELIVERIES_lo]

original_df['out_EARLY_DELIVERIES'].replace(to_replace = condition_hi,
                                 value      = 1,
                                 inplace    = True)

original_df['out_EARLY_DELIVERIES'].replace(to_replace = condition_lo,
                                 value      = 1,
                                 inplace    = True)


# LATE_DELIVERIES
original_df['out_LATE_DELIVERIES'] = 0
condition_hi = original_df.loc[0:,'out_LATE_DELIVERIES'][original_df['LATE_DELIVERIES'] > LATE_DELIVERIES_hi]

original_df['out_LATE_DELIVERIES'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

# AVG_PREP_VID_TIME
original_df['out_AVG_PREP_VID_TIME'] = 0
condition_hi = original_df.loc[0:,'out_AVG_PREP_VID_TIME'][original_df['AVG_PREP_VID_TIME'] > AVG_PREP_VID_TIME_hi]

original_df['out_AVG_PREP_VID_TIME'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


# AVG_CLICKS_PER_VISIT
original_df['out_AVG_CLICKS_PER_VISIT'] = 0
condition_lo = original_df.loc[0:,'out_AVG_CLICKS_PER_VISIT'][original_df['AVG_CLICKS_PER_VISIT'] > AVG_CLICKS_PER_VISIT_lo]

original_df['out_AVG_CLICKS_PER_VISIT'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# TOTAL_PHOTOS_VIEWED
original_df['out_TOTAL_PHOTOS_VIEWED'] = 0
condition_hi = original_df.loc[0:,'out_TOTAL_PHOTOS_VIEWED'][original_df['TOTAL_PHOTOS_VIEWED'] > TOTAL_PHOTOS_VIEWED_hi]

original_df['out_TOTAL_PHOTOS_VIEWED'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)


In [6]:
#declaring explanatory variables 
x_variables ={
    #original variables used to build the model
    
    'Original' : ['TOTAL_MEALS_ORDERED', 'CONTACTS_W_CUSTOMER_SERVICE','AVG_TIME_PER_SITE_VISIT',
                  'CANCELLATIONS_BEFORE_NOON','CANCELLATIONS_AFTER_NOON','TASTES_AND_PREFERENCES',
                  'PC_LOGINS','MOBILE_LOGINS', 'WEEKLY_PLAN','EARLY_DELIVERIES','LATE_DELIVERIES',
                  'PACKAGE_LOCKER','REFRIGERATED_LOCKER','FOLLOWED_RECOMMENDATIONS_PCT','AVG_PREP_VID_TIME',
                  'LARGEST_ORDER_SIZE','MASTER_CLASSES_ATTENDED','MEDIAN_MEAL_RATING','AVG_CLICKS_PER_VISIT',
                  'TOTAL_PHOTOS_VIEWED','out_TOTAL_MEALS_ORDERED','out_UNIQUE_MEALS_PURCH',
                  'out_CONTACTS_W_CUSTOMER_SERVICE','out_AVG_TIME_PER_SITE_VISIT','out_CANCELLATIONS_BEFORE_NOON',
                  'out_EARLY_DELIVERIES','out_LATE_DELIVERIES','out_AVG_PREP_VID_TIME'],
    
    #significant variables after logistic regression
    'Significant' : ['CANCELLATIONS_BEFORE_NOON', 'TASTES_AND_PREFERENCES',
                     'MOBILE_LOGINS','FOLLOWED_RECOMMENDATIONS_PCT']
}

# declaring response variables and explanatory variable

chef_data   =  original_df.loc[ : , x_variables['Significant']]
chef_target =  original_df.loc[ : , 'CROSS_SELL_SUCCESS']


#preparing the training and testing data
X_train,X_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            test_size    = 0.25,
            random_state = 222,
            stratify     = chef_target)



In [7]:
#Instantiating the full tree classification model
Chef_tree2  = DecisionTreeClassifier(max_depth = 4,
                                     min_samples_leaf = 25,
                                     random_state = 802)


#Fitting the model on the training data
Chef_tree2_fit  = Chef_tree2.fit(X_train, y_train)


#Predicting on new data using full tree classification model
Chef_tree2_pred = Chef_tree2_fit.predict(X_test)

#Scoring the results of the model
pruned_tree_train = Chef_tree2_fit.score(X_train, y_train).round(4)
pruned_tree_test = Chef_tree2_fit.score(X_test, y_test).round(4)
pruned_tree_AUC = roc_auc_score(y_true  = y_test, y_score = Chef_tree2_pred).round(4)

#Printing the scoring results of the model
print('Training ACCURACY:', Chef_tree2_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', Chef_tree2_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test, y_score = Chef_tree2_pred).round(4))




Training ACCURACY: 0.7868
Testing  ACCURACY: 0.7721
AUC Score        : 0.7612
